In [1]:
#data manipulation
import numpy as np
import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
import nltk
from nltk.tokenize import word_tokenize

#stats/ml
import scipy
import sklearn as sk
from scipy.stats import ttest_1samp
from scipy.stats import boxcox
from scipy.stats import chisquare
from sklearn.preprocessing import OneHotEncoder
#vis
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')

#additional support packages
from datetime import date, datetime, timedelta
import random
import re
import os
nltk.download('punkt')
os.cpu_count()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\notal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


16

In [2]:
def bin_small(df,col,replace_val,threshold=100,**kwargs):
    more_removal = [i for i,x in dict(df[col].value_counts()).items() if x<threshold]
    df[col]=np.where(df[col].isin(more_removal),replace_val,df[col])

<a id='top'></a>
# Overview

1. [Data Cleaning](#sec-1)
- Nulls
- Data inconsistencies
    
    
2. [Feature Engineering](#sec-2)
- 5 new features


3. [Additional Exploratory Data Analysis (EDA)](#sec-3)
- Univariate analysis of dependent variable
- Exploring independent variables relationships with each other and dependent variable

<a id='sec-1'></a>
## 1. Data Cleaning

In [3]:
data = pd.read_csv(r'..\data\adopt.csv')
legacy_data=data.copy()
data

,Pet ID,Outcome Type,Sex upon Outcome,Name,Found Location,Intake Type,Intake Condition,Pet Type,Sex upon Intake,Breed,Color,Age upon Intake Days,Time at Center
0,A745079,Transfer,Unknown,NaN,7920 Old Lockhart in Travis (TX),Stray,Normal,Cat,Unknown,Domestic Shorthair Mix,Blue,3,0
1,A801765,Transfer,Intact Female,NaN,5006 Table Top in Austin (TX),Stray,Normal,Cat,Intact Female,Domestic Shorthair,Brown Tabby/White,28,0
2,A667965,Transfer,Neutered Male,NaN,14100 Thermal Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,Chihuahua Shorthair Mix,Brown/Tan,1825,0
3,A687551,Transfer,Intact Male,NaN,5811 Cedardale Dr in Austin (TX),Stray,Normal,Cat,Intact Male,Domestic Shorthair Mix,Brown Tabby,28,0
4,A773004,Adoption,Neutered Male,*Boris,Highway 290 And Arterial A in Austin (TX),Stray,Normal,Dog,Intact Male,Chihuahua Shorthair Mix,Tricolor/Cream,365,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95379,A706720,Adoption,Neutered Male,Nikko,Mc Callen Pass And Parmer in Austin (TX),Stray,Normal,Dog,Neutered Male,Miniature Schnauzer Mix,Tan/Gray,1460,0
95380,A782751,Adoption,Neutered Male,NaN,18706 Blake Manor Rd in Manor (TX),Stray,Normal,Dog,Intact Male,American Pit Bull Terrier Mix,Brown,60,0
95381,A768058,Euthanasia,Unknown,NaN,1701 Congress Avenue in Austin (TX),Wildlife,Normal,Other,Unknown,Bat Mix,Black/Black,730,0
95382,A729326,Adoption,Neutered Male,*Jester,5017 W. 290 in Austin (TX),Stray,Normal,Dog,Intact Male,Pointer Mix,Black/White,730,0


In [4]:
data['Outcome Type'].value_counts()

Adoption           39734
Transfer           32299
Return to Owner    13719
Euthanasia          7630
Died                1053
Disposal             503
Rto-Adopt            369
Missing               52
Relocate              20
Name: Outcome Type, dtype: int64

In [5]:
data['Time at Center'].value_counts()[1]/(data['Time at Center'].value_counts().sum())

0.0874045961586849

In [6]:
data['Time at Center'].value_counts()

0    87047
1     8337
Name: Time at Center, dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95384 entries, 0 to 95383
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Pet ID                95384 non-null  object
 1   Outcome Type          95379 non-null  object
 2   Sex upon Outcome      95383 non-null  object
 3   Name                  59093 non-null  object
 4   Found Location        95384 non-null  object
 5   Intake Type           95384 non-null  object
 6   Intake Condition      95384 non-null  object
 7   Pet Type              95384 non-null  object
 8   Sex upon Intake       95383 non-null  object
 9   Breed                 95384 non-null  object
 10  Color                 95384 non-null  object
 11  Age upon Intake Days  95384 non-null  int64 
 12  Time at Center        95384 non-null  int64 
dtypes: int64(2), object(11)
memory usage: 9.5+ MB


### KEY POINTS IDENTIFIED:
    
1. **Primary key - "Pet ID" column**
    - We will check for uniqueness later


1. **4 columns with inconsistent non-null counts**
    - Name - 59093 count
    - Outcome - 95379 count
    - Sex upon Outcome - 95383 count
    - Sex upon Intake - 95383 count
    
    
3. **One integer, independent variable**
    - Age (days), discrete values 
    
    
4. **10 objects, independent variables**
    - Categorical
    - *Note: "Name" weak as categorical. Given the high nulls and high uniqueness of names, it is likely we may have to drop name as a feature, will explore further*
    
    
5. **Dependent variable: "Time at Center"**

In [8]:
data.describe()

,Age upon Intake Days,Time at Center
count,95384.000000,95384.000000
mean,704.154691,0.087405
std,1052.579518,0.282429
min,0.000000,0.000000
25%,30.000000,0.000000
50%,365.000000,0.000000
75%,730.000000,0.000000
max,9125.000000,1.000000


"Time at Center" can be ignored because it is a boolean masked as an integer column. 

"Age upon Intake Days" the standard deviation is larger than the mean value. This suggests there are some large outliers on the upper bound. We will investigate what steps need to be taken for this column in the section 3 of this notebook

### Checking Primary Key column

In [9]:
data['Pet ID'].value_counts().max()

1

"Pet ID" column is indeed unique. Let's make sure all columns contain the same data type

### Checking for mixed data type columns

In [10]:
for i in data.columns.values:
    type_array = data[i].apply(type).value_counts()
    if len(type_array)>1:
        print(f'{type_array}')
        print()
    else:
        pass

<class 'str'>      95379
<class 'float'>        5
Name: Outcome Type, dtype: int64

<class 'str'>      95383
<class 'float'>        1
Name: Sex upon Outcome, dtype: int64

<class 'str'>      59093
<class 'float'>    36291
Name: Name, dtype: int64

<class 'str'>      95383
<class 'float'>        1
Name: Sex upon Intake, dtype: int64



All columns listed above are columns with null values based on the info method we passed earlier. The additional floating point values are nulls. We're going to take a look at "Outcome Type", "Sex upon Outcome" and "Sex upon Intake" first because the number of null values is low. We should easily be able to determine what to do with these null values. Then we will take a look at "Name"

### Handling "Outcome Type" nulls

In [11]:
data['Outcome Type'].value_counts()

Adoption           39734
Transfer           32299
Return to Owner    13719
Euthanasia          7630
Died                1053
Disposal             503
Rto-Adopt            369
Missing               52
Relocate              20
Name: Outcome Type, dtype: int64

In [12]:
data.loc[data['Outcome Type'].isnull()]

,Pet ID,Outcome Type,Sex upon Outcome,Name,Found Location,Intake Type,Intake Condition,Pet Type,Sex upon Intake,Breed,Color,Age upon Intake Days,Time at Center
828,A674209,NaN,Unknown,NaN,4210 W. Braker in Austin (TX),Wildlife,Normal,Other,Unknown,Bat Mix,Brown,0,0
10178,A679066,NaN,Unknown,NaN,Austin (TX),Wildlife,Normal,Other,Unknown,Rabbit Sh,Brown,28,0
18664,A686025,NaN,Unknown,NaN,12901 N Ih 35 #1100 in Austin (TX),Wildlife,Normal,Other,Unknown,Bat Mix,Brown,365,0
47605,A671017,NaN,Unknown,NaN,10105 Falcon Ridge Dr. in Travis (TX),Wildlife,Sick,Other,Unknown,Fox Mix,Brown/Black,365,0
58378,A815725,NaN,Unknown,NaN,321 West Ben White in Austin (TX),Wildlife,Normal,Other,Unknown,Bat,Black,365,0


Based on the values in "Outcome Type" and the number of nulls is low (5), I don't think imputing the values or replacing it with a dummy category like "blank" is necessary and we can likely remove these values. We just need to check these nulls are not associated with rare information in other columns.

Check
- Unknown in "Sex upon Outcome" and "Sex upon Intake" is not associated only with null outcome type
- "Name" has many nulls so we do not have to check
- "Found Location", perhaps City, Province will need to be extracted and checked
- "Intake Type" ensure Wildlife is not associated only with null outcome type
- "Pet Type" ensure Other is not associated with only null outcome type
- "Breed" seems mixed, double check bat info is not under-represented after removal
- "Age upon Intake Days" ensure distribution is not affected from removal of 0,28,365,365,365
- "Time at Center" ensure dependent variable not largely affected by removal

In [13]:
for i in ['Sex upon Outcome','Intake Type','Intake Condition','Sex upon Intake','Pet Type']:
    print(data[i].value_counts())
    print()

Neutered Male    30233
Spayed Female    28123
Intact Female    13686
Intact Male      13614
Unknown           9727
Name: Sex upon Outcome, dtype: int64

Stray                 70112
Owner Surrender       15131
Public Assist          5236
Wildlife               4559
Euthanasia Request      235
Abandoned               111
Name: Intake Type, dtype: int64

Normal      81824
Injured      5384
Sick         4289
Nursing      3163
Aged          352
Other         189
Feral          97
Pregnant       63
Medical        21
Behavior        2
Name: Intake Condition, dtype: int64

Intact Male      33326
Intact Female    32455
Neutered Male    10521
Unknown           9727
Spayed Female     9354
Name: Sex upon Intake, dtype: int64

Dog          48674
Cat          40023
Other         6118
Bird           553
Livestock       16
Name: Pet Type, dtype: int64



In [14]:
data.loc[data.Breed.str.contains('Bat')].shape[0]

3030

Based on these values, removal of the 5 nulls from "Outcome Type" will not impact the dataset. Now to look at "Sex upon Outcome".

In [15]:
data=data.loc[data['Outcome Type'].notnull()] #removing nulls from Outcome Type

### Handling "Sex upon Outcome" and "Sex upon Intake" fields

In [16]:
data.loc[data['Sex upon Outcome'].isnull()] 

,Pet ID,Outcome Type,Sex upon Outcome,Name,Found Location,Intake Type,Intake Condition,Pet Type,Sex upon Intake,Breed,Color,Age upon Intake Days,Time at Center
38480,A667395,Return to Owner,NaN,Diego,Pflugerville (TX),Owner Surrender,Normal,Dog,NaN,Dachshund,Brown Merle,2555,0


Based on analysis of previous columns, we can include this record by replacing the null values with "Unknown" in both gender related columns. Unknown is already used to fill missing values in these columns.

In [17]:
data['Sex upon Outcome'].fillna('Unknown',inplace=True)
data['Sex upon Intake'].fillna('Unknown',inplace=True)

C:\Users\notal\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## Dealing with Name

Because almost 40% of the data has null name values, we will not be removing them. Instead, we will replace the values with "Unknown" to stay consistent with other column nomenclature for null values.


In [18]:
data['Name'].fillna('Unknown',inplace=True)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95379 entries, 0 to 95383
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Pet ID                95379 non-null  object
 1   Outcome Type          95379 non-null  object
 2   Sex upon Outcome      95379 non-null  object
 3   Name                  95379 non-null  object
 4   Found Location        95379 non-null  object
 5   Intake Type           95379 non-null  object
 6   Intake Condition      95379 non-null  object
 7   Pet Type              95379 non-null  object
 8   Sex upon Intake       95379 non-null  object
 9   Breed                 95379 non-null  object
 10  Color                 95379 non-null  object
 11  Age upon Intake Days  95379 non-null  int64 
 12  Time at Center        95379 non-null  int64 
dtypes: int64(2), object(11)
memory usage: 10.2+ MB


Now all columns in our data set have the same number of non-null values. We have removed 5 rows from the entire dataset from this cleaning step

## Ensuring data consistencies
- Noticed "*" prefix to certain names - need to clean this up
- Check other columns for formatting inconsistencies

### "Name" cleaning

In [20]:
data['Name']=data['Name'].str.replace('[*]','',regex=True) #remove asteriks
print(len(data.Name.unique()))
data.Name.value_counts()

14107


<ipython-input-20-719e600a3ca0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Name']=data['Name'].str.replace('[*]','',regex=True) #remove asteriks


Unknown        36304
Bella            370
Max              359
Luna             344
Charlie          305
               ...  
Leti               1
Bodan              1
Mr Meeseeks        1
Gadget             1
Boswell            1
Name: Name, Length: 14107, dtype: int64

### Breed check

Because the categorical size of  "Livestock" are is small, we will bin it together with "Other". The model will not be able to generalize correctly from such a small bin. "Bird" may be a bit small, but it may be relevant.

In [21]:
data['Pet Type']=data['Pet Type'].replace(to_replace={'Livestock':'Other'})

<ipython-input-21-09d109471959>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Pet Type']=data['Pet Type'].replace(to_replace={'Livestock':'Other'})


I was stumped on the best way to reduce the number of bins in this category and just thought of using tokenization to reduce binning further. Other approaches I considered.

- Thresholding by mean or percentile values
    - *Issue: Even binning all categories under 600 in size led to 20k records with undefined breeds - a significant loss of information*
- Thresholding by a proportion of the minority class
    - e.g. Bins must contain 20% of minority class size
- Extract bi-grams to derive smaller groups
- Using Levenshtein distance to organize categories by edit distance
    
    
First we will bin the breeds of smaller "Pet Type" categories like "Bird" and "Other".


In [22]:
data['Mix']=np.where((data['Breed'].str.contains('Mix'))|(data['Breed'].str.contains('/')),"Mix","Not")

data['Breed_new']=data['Breed'].str.replace('Mix','')
data['Breed_new']=data['Breed_new'].str.replace(r'[/]',' ',regex=True)
data['Breed_new']=data['Breed_new'].str.lower()
data['Breed_new'].value_counts()

<ipython-input-22-35038a1d5fe8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Mix']=np.where((data['Breed'].str.contains('Mix'))|(data['Breed'].str.contains('/')),"Mix","Not")
<ipython-input-22-35038a1d5fe8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Breed_new']=data['Breed'].str.replace('Mix','')
<ipython-input-22-35038a1d5fe8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

domestic shorthair                 27654
domestic shorthair                  5068
pit bull                            5013
chihuahua shorthair                 4960
labrador retriever                  4783
                                   ...  
chesa bay retr pit bull                1
snowshoe domestic shorthair            1
deer                                   1
basset hound golden retriever          1
afghan hound labrador retriever        1
Name: Breed_new, Length: 2395, dtype: int64

In [23]:
#token approach
breed_dict={}
for i in data['Breed_new'].values:
    salad = word_tokenize(i)
    for s in salad:
        if s in breed_dict.keys():
            breed_dict[s]=breed_dict[s]+1
        else:
            breed_dict[s]=1
breed_list=[[i,x] for i,x in breed_dict.items()]
test=pd.DataFrame(data=breed_list,columns=['Token','Count'])
test = test.sort_values(by='Count',ascending=False)

In [24]:
len(test.Token.unique())

406

In [27]:

for i in test.Token.values[:30]:
    data['Breed_'+i]=np.where(data['Breed_new'].str.contains(i),1,0)


<ipython-input-27-1e67a079603e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Breed_'+i]=np.where(data['Breed_new'].str.contains(i),1,0)


In [28]:
data['MixColor']=np.where((data['Color'].str.contains('Mix'))|(data['Color'].str.contains('/')),"Mix","Not")

data['Color_new']=data['Color'].str.replace('Mix','')
data['Color_new']=data['Color_new'].str.replace(r'[/]',' ',regex=True)
data['Color_new']=data['Color_new'].str.lower()
data['Color_new'].value_counts()

<ipython-input-28-4ed5672bcb91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MixColor']=np.where((data['Color'].str.contains('Mix'))|(data['Color'].str.contains('/')),"Mix","Not")
<ipython-input-28-4ed5672bcb91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Color_new']=data['Color'].str.replace('Mix','')
<ipython-input-28-4ed5672bcb91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

black white           9671
black                 8514
brown tabby           6069
brown                 4439
white                 3310
                      ... 
brown tabby blue         1
tan cream tabby          1
tan yellow brindle       1
fawn cream               1
red merle tan            1
Name: Color_new, Length: 566, dtype: int64

In [29]:
#token approach
breed_dict={}
for i in data['Color_new'].values:
    salad = word_tokenize(i)
    for s in salad:
        if s in breed_dict.keys():
            breed_dict[s]=breed_dict[s]+1
        else:
            breed_dict[s]=1
Color_list=[[i,x] for i,x in breed_dict.items()]
test=pd.DataFrame(data=Color_list,columns=['Token','Count'])
test = test.sort_values(by='Count',ascending=False)

In [30]:

for i in test.Token.values[:20]:
    data['Color_'+i]=np.where(data['Color_new'].str.contains(i),1,0)


<ipython-input-30-b98bd19e1432>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Color_'+i]=np.where(data['Color_new'].str.contains(i),1,0)


In [31]:
data['Intake Condition']=data['Intake Condition'].replace(to_replace={'Behavior':'Other','Medical':'Injured'})

<ipython-input-31-cf7cf1225bbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Intake Condition']=data['Intake Condition'].replace(to_replace={'Behavior':'Other','Medical':'Injured'})


Color reduced to 25 bins. Split into two features - mix color and base color. 

Next steps:
- Clean out deprecated columns
- Engage in additional feature engineering

In [32]:
data.rename(columns={'Mix':'MixBreed'},inplace=True) #standardize naming of columns

C:\Users\notal\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


<a id='sec-2'></a>
## 2. Feature Engineering

Ideas:
- Spay/Neuter status
- Spay/Neuter change status (at shelter)
- Location by City/State
- Name starting letter
- Name length


[*Back to top*](#top)

In [33]:
data['NS_intake']=np.where((data['Sex upon Intake'].str.contains('Neutered'))
                          |(data['Sex upon Intake'].str.contains('Spayed')), 1,0) #create boolean infertility on intake
data['Gender']=data['Sex upon Intake'].str.split(' ').str[-1] #new feature

print(data.NS_intake.value_counts())
print(data.Gender.value_counts())

<ipython-input-33-2f30cd3411fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NS_intake']=np.where((data['Sex upon Intake'].str.contains('Neutered'))
<ipython-input-33-2f30cd3411fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Gender']=data['Sex upon Intake'].str.split(' ').str[-1] #new feature


0    75504
1    19875
Name: NS_intake, dtype: int64
Male       43847
Female     41809
Unknown     9723
Name: Gender, dtype: int64


In [34]:
data['Sex upon Outcome'].value_counts()

Neutered Male    30233
Spayed Female    28123
Intact Female    13686
Intact Male      13614
Unknown           9723
Name: Sex upon Outcome, dtype: int64

In [35]:
data['NS_clinic']=np.where((data.NS_intake==0)&((data['Sex upon Outcome'].str.contains('Neutered'))
                          |(data['Sex upon Outcome'].str.contains('Spayed'))),1,0) #new feature
data['NS_clinic'].value_counts()

<ipython-input-35-cd7c77a7f7c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NS_clinic']=np.where((data.NS_intake==0)&((data['Sex upon Outcome'].str.contains('Neutered'))


0    56898
1    38481
Name: NS_clinic, dtype: int64

In [36]:
data['Found Location']

0                 7920 Old Lockhart in Travis (TX)
1                    5006 Table Top in Austin (TX)
2                  14100 Thermal Dr in Austin (TX)
3                 5811 Cedardale Dr in Austin (TX)
4        Highway 290 And Arterial A in Austin (TX)
                           ...                    
95379     Mc Callen Pass And Parmer in Austin (TX)
95380           18706 Blake Manor Rd in Manor (TX)
95381          1701 Congress Avenue in Austin (TX)
95382                   5017 W. 290 in Austin (TX)
95383                                  Austin (TX)
Name: Found Location, Length: 95379, dtype: object

We could input these into map api and get longitude and latitudinal data, or zip code and regionalize info.

We will just extract second last element in string (city)

In [37]:
data['City']=data['Found Location'].str.split(' ').str[-2]
data.City.value_counts()

<ipython-input-37-96ecfce2efe6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['City']=data['Found Location'].str.split(' ').str[-2]


Austin          79474
Travis           8442
Manor            2069
Valle            1979
Pflugerville     1547
Outside           605
Leander           245
Vista             166
Lakeway           129
Creedmoor         121
Park              113
Jonestown          92
Ridge              72
Webberville        55
Bastrop            53
Hays               45
Rock               41
Cave               41
Williamson         27
Hills              20
Valley             14
Venture            12
Caldwell            5
Rollingwood         5
Burnet              3
Hill                1
Leanna              1
Blanco              1
Name: City, dtype: int64

In [38]:
bin_small(data,'City','Other City',threshold=1000)
data.City.value_counts() #new feature

<ipython-input-2-da11fa3d6af4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col]=np.where(df[col].isin(more_removal),replace_val,df[col])


Austin          79474
Travis           8442
Manor            2069
Valle            1979
Other City       1867
Pflugerville     1547
Name: City, dtype: int64

In [39]:
data['FirstLetterName']=np.where(data.Name!='Unknown',data.Name.str[0],'Unknown')
data['FirstLetterName']=np.where(data.FirstLetterName.str.isalpha(),data.FirstLetterName,'ID')
data['FirstLetterName']=data.FirstLetterName.replace(to_replace={'É':'E'})
data['FirstLetterName'].fillna('Unknown',inplace=True)
data['FirstLetterName'].value_counts() #new feature

<ipython-input-39-fed8aa445911>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FirstLetterName']=np.where(data.Name!='Unknown',data.Name.str[0],'Unknown')
<ipython-input-39-fed8aa445911>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FirstLetterName']=np.where(data.FirstLetterName.str.isalpha(),data.FirstLetterName,'ID')
<ipython-input-39-fed8aa445911>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

Unknown    36306
B           5783
S           5708
M           5353
C           5294
L           3903
P           3902
R           3294
T           3050
D           2705
J           2619
A           2586
G           2025
H           1895
F           1670
K           1617
E           1333
N           1301
O           1272
W           1132
Z            788
I            467
V            460
ID           354
Y            201
Q            165
X            114
U             82
Name: FirstLetterName, dtype: int64

In [40]:
data['NameLength']=data['Name'].map(lambda x: len(x))
data['NameLength'].value_counts()

<ipython-input-40-9586a5da62f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NameLength']=data['Name'].map(lambda x: len(x))


7     43083
5     16785
6     12925
4     11854
8      3330
3      2983
9      1848
10      976
11      651
12      390
2       344
1        76
13       42
14       28
15       24
16       10
17        9
20        8
18        6
21        3
0         2
19        1
23        1
Name: NameLength, dtype: int64

In [41]:
nl = []
for i in data.iterrows():
    if i[1].Name=='Unknown':
        nl.append('Unknown')
    else:
        if i[1].NameLength==0:
            nl.append('Unknown')
        elif (i[1].NameLength>=1)&(i[1].NameLength<4):
            nl.append('short')
        elif (i[1].NameLength>=1)&(i[1].NameLength<8):
            nl.append('med')
        else:
            nl.append('long')
data['NameLengthBin']=nl #new feature
data['NameLengthBin'].value_counts()

<ipython-input-41-3a5df900504a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NameLengthBin']=nl #new feature


med        48343
Unknown    36306
long        7327
short       3403
Name: NameLengthBin, dtype: int64

After adding 5 new features, let's remove depreciated columns again and move on to EDA and see if any of these additional features and original features hold any value

In [42]:
#additional cleaning
bin_small(data,'Outcome Type','Other Outcome',threshold=2000)
bin_small(data,'Intake Condition','Other')
data['City'].fillna('Other City',inplace=True)

<ipython-input-2-da11fa3d6af4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col]=np.where(df[col].isin(more_removal),replace_val,df[col])


In [43]:
cat=['Outcome Type', 'Intake Type', 'Intake Condition',
'Pet Type', 'MixBreed','MixColor',  'NS_intake', 'NS_clinic',
'City', 'Gender', 'FirstLetterName', 'NameLengthBin',]
data=pd.get_dummies(data,drop_first=True,columns=cat) #one hot encode categorical variables
# data.drop(cat,axis=1,inplace=True)


In [44]:
data.columns.values

array(['Pet ID', 'Sex upon Outcome', 'Name', 'Found Location',
       'Sex upon Intake', 'Breed', 'Color', 'Age upon Intake Days',
       'Time at Center', 'Breed_new', 'Breed_shorthair', 'Breed_domestic',
       'Breed_retriever', 'Breed_labrador', 'Breed_chihuahua',
       'Breed_bull', 'Breed_pit', 'Breed_terrier', 'Breed_shepherd',
       'Breed_german', 'Breed_hair', 'Breed_medium', 'Breed_australian',
       'Breed_bat', 'Breed_longhair', 'Breed_miniature',
       'Breed_dachshund', 'Breed_dog', 'Breed_cattle', 'Breed_poodle',
       'Breed_border', 'Breed_collie', 'Breed_siamese', 'Breed_american',
       'Breed_boxer', 'Breed_great', 'Breed_husky', 'Breed_hound',
       'Breed_beagle', 'Breed_raccoon', 'Color_new', 'Color_white',
       'Color_black', 'Color_brown', 'Color_tabby', 'Color_tan',
       'Color_blue', 'Color_orange', 'Color_gray', 'Color_red',
       'Color_tricolor', 'Color_brindle', 'Color_cream', 'Color_tortie',
       'Color_calico', 'Color_point', 'Color_choco

In [45]:
data.drop(['Sex upon Outcome', 'Name', 'Found Location',
       'Sex upon Intake', 'Breed', 'Color', 'Age upon Intake Days','Breed_new','Color_new',],axis=1,inplace=True)
data.to_csv(r'..\data\feature3.csv',index=False)


# END

[*Back to top*](#top)